#### 작성자 : 서연호 
* Kakao Crawler

In [15]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time
from tqdm import tqdm
import pandas as pd

In [16]:
driver_path = 'C:/dev_python/Webdriver/chromedriver' #드라이버 경로 설정
url = 'https://map.kakao.com/'                       #url 설정
query ='김천숙박'                                    #검색어 입력

In [17]:
driver = webdriver.Chrome(driver_path)
driver.get(url)

<ipython-input-17-3b45c425858d>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


### Query Search

In [18]:
query_p = '#search\.keyword\.query'
query_enter_p ='#search\.keyword\.submit'

try :
    driver.find_element_by_css_selector(query_p).send_keys(query)
    driver.find_element_by_css_selector(query_enter_p).send_keys(Keys.RETURN)
except NoSuchElementException :
    time.sleep(2)
    driver.find_element_by_css_selector(query_p).send_keys(query)
    driver.find_element_by_css_selector(query_enter_p).send_keys(Keys.RETURN)
    
time.sleep(1)

<ipython-input-18-eeb5d48f19fa>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector(query_p).send_keys(query)
<ipython-input-18-eeb5d48f19fa>:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector(query_enter_p).send_keys(Keys.RETURN)


### Data Info

In [19]:
#data-List
place_names = []
category = []
urls = []
review_urls = []
address = []
stars = []
stars_counts = []
tels = []

#data_path 
place_names_p = '#info\.search\.place\.list > li > div.head_item.clickArea > strong > a.link_name'
category_p ='#info\.search\.place\.list > li > div.head_item.clickArea > span'
addres_p = '#info\.search\.place\.list > li > div.info_item > div.addr > p.lot_number'
stars_p = '#info\.search\.place\.list > li > div.rating.clickArea > span.score > em'
stars_counts_p = '#info\.search\.place\.list > li > div.rating.clickArea > span.score > a'
tels_p = '#info\.search\.place\.list > li > div.info_item > div.contact.clickArea > span.phone'
urls_p = '#info\.search\.place\.list > li > div.info_item > div.contact.clickArea > a.homepage'
review_urls_p='#info\.search\.place\.list > li > div.rating.clickArea > span.score > a'

### 매장 정보 크롤링 

In [20]:
search_count = 20 # 한페이지당 15개 정보 , 크롤링할 페이지수 설정
page_counter = 2  # 1~5 까지 변경되며 페이지를 클릭하기 위한 변수

# 더보기 클릭 (클릭시 2페이지에서 부터 시작)
driver.find_element_by_css_selector('a#info\.search\.place\.more').send_keys(Keys.RETURN)

for i in tqdm(range(2,int(search_count)+1)) :
    time.sleep(1)
    
    # 스크롤 하는 코드 구현필요 
    # 구현위치 
    
    if (i-1) % 5 == 0 :#페이지수 더보기 > 버튼 클릭 
        try :
            driver.find_element_by_css_selector('#info\.search\.page\.next').send_keys(Keys.RETURN)
            page_counter = 1

        except NoSuchElementException : 
            break
       
    if i == 2 :        
        page_counter = 3
    
    elif  page_counter == 1 : 
        page_counter = page_counter + 1
        pass 
   
    elif  i != 2 :
        try :
            page_element = 'a#info\.search\.page\.no{}'.format(str(page_counter))
            driver.find_element_by_css_selector(page_element).send_keys(Keys.RETURN)
            page_counter = page_counter + 1
        except :
            break

    time.sleep(2)
    
    
    # Text로 변경하기 위한 임시 리스트 변수 
    place_names_temp = driver.find_elements_by_css_selector(place_names_p)
    category_temp = driver.find_elements_by_css_selector(category_p)
    address_temp = driver.find_elements_by_css_selector(addres_p)
    stars_temp = driver.find_elements_by_css_selector(stars_p)
    stars_cnt_temp = driver.find_elements_by_css_selector(stars_counts_p)
    tels_temp = driver.find_elements_by_css_selector(tels_p)
    url_temp = driver.find_elements_by_css_selector(urls_p)
    review_urls_temp = driver.find_elements_by_css_selector(review_urls_p)
    
    #List -> text 변환
    for index in range(len(place_names_temp)) :
        
        name_t = place_names_temp[index].text
        cate_t = category_temp[index].text
        
        # ex (지번) 불로동 361-4  -> 불로동 361-4 로 (지번) 삭제 
        try :
            address_t = address_temp[index].text.split(')')[1].strip()
        except : 
            address_t = None
                
        stars_t = stars_temp[index].text 
        
        # ex 5건 -> '건' 삭제
        stars_cnt_t = stars_cnt_temp[index].text.split('건')[0].strip()
        # -> 향후 리뷰 0건인 매장은 리뷰정보를 수집하지 않을 예정으로 integer type 으로 변환 후 저장 
        stars_cnt_int = int(stars_cnt_t)
        
        tels_t = tels_temp[index].text
        if tels_t == '' :
            tels_t = None
            
        #홈페이지 미지정시 url https://map.kakao.com/#none 으로 확인 -> https://map.kakao.com/#none 경우 None 으로 저장 
        url_t = url_temp[index].get_attribute('href')   
        if url_t =='https://map.kakao.com/#none' :
            url_t = None
            
        review_t =review_urls_temp[index].get_attribute('href')
        
        # 변환된 text -> 본래 리스트 변수의 저장
        place_names.append(name_t)
        category.append(cate_t)
        urls.append(url_t)
        review_urls.append(review_t)
        address.append(address_t)
        stars.append(stars_t)
        stars_counts.append(stars_cnt_int)
        tels.append(tels_t)
        
        #print()
        #print('--------------------------------')
        #print('이름 : ',name_t)
        #print('주소 : ',address_t)
        #print('카테고리 : ',cate_t)
        #print('url : ',url_t)
        #print('review-url : ',review_t)
        #print('리뷰 개수 : ',stars_cnt_t)
        #print('리뷰 평점 : ',stars_t)
        #print('번호 : ',tels_t)
        #print('--------------------------------')


<ipython-input-20-17cac7f035e3>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('a#info\.search\.place\.more').send_keys(Keys.RETURN)
  0%|                                                                                           | 0/19 [00:00<?, ?it/s]<ipython-input-20-17cac7f035e3>:37: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  place_names_temp = driver.find_elements_by_css_selector(place_names_p)
<ipython-input-20-17cac7f035e3>:38: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  category_temp = driver.find_elements_by_css_selector(category_p)
<ipython-input-20-17cac7f035e3>:39: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  address_temp = driver.find_elements_by_css_selector(addres_p)
<ipython-input-20-17cac7f035e3>:40: DeprecationW

In [21]:
kakaomaps_df = pd.DataFrame({
    'place':place_names,
    'category':category,
    'address':address,
    'tel':tels,
    'review_count':stars_counts,
    'total star':stars,
    'url':urls
})

In [22]:
kakaomaps_df

,place,category,address,tel,review_count,total star,url
0,B&B모텔,"여관,모텔",부곡동 424-3,054-910-1199,5,1.6,https://kcbnb.modoo.at/
1,해인산장,"산장,별장",부항면 해인리 266,054-437-1991,1,5.0,http://www.xn--zf4b17j5dm97c.com/
2,캠핑뜨락오토캠핑장,오토캠핑장,조마면 강곡리 917-1,010-4114-7122,0,0.0,http://cafe.naver.com/campingdderock
3,베네치아,"여관,모텔",감호동 11-2,054-436-1089,1,1.0,None
4,하늘아래첫동네오토캠핑장,오토캠핑장,증산면 금곡리 801,None,0,0.0,http://cafe.naver.com/hahadongcamping
...,...,...,...,...,...,...,...
268,퍼피플,숙박,상촌면 하도대리 25-17,None,0,0.0,None
269,바른품미숯불구이원호점,숙박,고아읍 원호리 448-1,None,0,0.0,None
270,또랑민박,민박,상촌면 대해리 1035-3,None,0,0.0,None
271,풀빛동산펜션,펜션,상촌면 대해리 381,None,0,0.0,None


### 리뷰정보 Data Info 

In [23]:
#data_list 
user_name = []
date = []
comment = []
user_stars = []

#data_path 
user_name_p ='#mArticle > div.cont_evaluation > div.evaluation_review > ul > li > div.comment_info > div > a.link_user'
date_p = '#mArticle > div.cont_evaluation > div.evaluation_review > ul > li > div.comment_info > div > span.time_write'
comment_p = '#mArticle > div.cont_evaluation > div.evaluation_review > ul > li > div.comment_info > p > span'
user_stars_p = '#mArticle > div.cont_evaluation > div.evaluation_review > ul > li > div.star_info > div > em'



### 리뷰정보 크롤링

In [24]:
for index in tqdm(range(len(stars_counts))) :
    if stars_counts[index] == 0 :
        comment.append(None)
        user_name.append(None)
        date.append(None)
        user_stars.append(None)
        continue 
    else :
        driver.get(review_urls[index])
        time.sleep(1)
        
    user_name_text = []
    date_text = []
    comment_text = []
    user_stars_text = []
    
    user_name_temp = driver.find_elements_by_css_selector(user_name_p)
    date_temp = driver.find_elements_by_css_selector(date_p)
    comment_temp = driver.find_elements_by_css_selector(comment_p)
    user_stars_temp = driver.find_elements_by_css_selector(user_stars_p)
    
    for index in range(len(user_name_temp)) :
        user_name_text.append(user_name_temp[index].text)
        date_text.append(date_temp[index].text)    
        comment_text.append(comment_temp[index].text)
        user_stars_text.append(user_stars_temp[index].text)
        
        #print()
        #print("=========================================")
        #print('userName : ',user_name_text)
        #print('writeDate : ',date_text)
        #print('comment : ',comment_text)
        #print('stars : ',comment_text)
        
    user_name.append(user_name_text)
    date.append(date_text)
    comment.append(comment_text)
    user_stars.append(user_stars_text)
    

  0%|                                                                                          | 0/273 [00:00<?, ?it/s]<ipython-input-24-91cb8e39773b>:17: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  user_name_temp = driver.find_elements_by_css_selector(user_name_p)
<ipython-input-24-91cb8e39773b>:18: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  date_temp = driver.find_elements_by_css_selector(date_p)
<ipython-input-24-91cb8e39773b>:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  comment_temp = driver.find_elements_by_css_selector(comment_p)
<ipython-input-24-91cb8e39773b>:20: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  user_stars_temp = driver.find_elements_by_css_selector(user_stars_p)
100%|████████████████████████████████████████████████████████████████████████

In [25]:
reviewInfo_df = pd.DataFrame({
    'user_name':user_name,
    'write_date':date,
    'user_stars':user_stars,
    'comment':comment
})

In [30]:
reviewInfo_df.head(10)

,user_name,write_date,user_stars,comment
0,"[YM, 임덕규, 앙기모띠주는나무, 천승민, 김준성]","[2021.06.28., 2020.05.27., 2020.04.15., 2020.0...","[1, 2, 1, 1, 3]","[진짜 불친절합니다, 모텔 옆에 사는 인간이 개를 키우는데 새벽에 개가 갑자기 엄청..."
1,[정발산기슭],[2021.02.06.],[5],[사장님도 친절하시고 고기가 너무 맛있어요. 고기뿐만 아니라 밑반찬도 너무 맛있네요...
2,None,None,None,None
3,[.],[2020.09.19.],[1],"[자는데 리모컨 누르지도 않았는데 갑자기 중앙등이 켜지고, 밤새도록 바람소리에 문..."
4,None,None,None,None
5,"[나만 당할 수 없다, ♡지민도♡]","[2020.07.06., 2019.07.28.]","[5, 5]",[여러 번 가보았지만 갈 때마다 한결같이 깨끗합니다. 특히나 수영장과 계곡은 너무 ...
6,[.],[2021.07.09.],[4],"[아기자기하고 아담하니, 편안한 캠핑장. 10+1 쿠폰이 있다]"
7,"[ᆞ쌤ᆞ, 😍, 쿤]","[2020.09.01., 2019.04.29., 2018.07.13.]","[5, 1, 5]",[어때 매진 됐으나 전화로 특실 예약...자정도착.vip제공 받음. 803호 스파 ...
8,[mmmm],[2021.10.16.],[5],[급하게 하루 자야해서 찾다가 온 곳 너무 깨끗하고 사장님도 친절하시고 황토방이라서...
9,None,None,None,None


### 매장 정보 , 리뷰정보 Merge

In [27]:
result = pd.concat([kakaomaps_df,reviewInfo_df],axis=1)

,place,category,address,tel,review_count,total star,url,user_name,write_date,user_stars,comment
0,B&B모텔,"여관,모텔",부곡동 424-3,054-910-1199,5,1.6,https://kcbnb.modoo.at/,"[YM, 임덕규, 앙기모띠주는나무, 천승민, 김준성]","[2021.06.28., 2020.05.27., 2020.04.15., 2020.0...","[1, 2, 1, 1, 3]","[진짜 불친절합니다, 모텔 옆에 사는 인간이 개를 키우는데 새벽에 개가 갑자기 엄청..."
1,해인산장,"산장,별장",부항면 해인리 266,054-437-1991,1,5.0,http://www.xn--zf4b17j5dm97c.com/,[정발산기슭],[2021.02.06.],[5],[사장님도 친절하시고 고기가 너무 맛있어요. 고기뿐만 아니라 밑반찬도 너무 맛있네요...
2,캠핑뜨락오토캠핑장,오토캠핑장,조마면 강곡리 917-1,010-4114-7122,0,0.0,http://cafe.naver.com/campingdderock,None,None,None,None
3,베네치아,"여관,모텔",감호동 11-2,054-436-1089,1,1.0,None,[.],[2020.09.19.],[1],"[자는데 리모컨 누르지도 않았는데 갑자기 중앙등이 켜지고, 밤새도록 바람소리에 문..."
4,하늘아래첫동네오토캠핑장,오토캠핑장,증산면 금곡리 801,None,0,0.0,http://cafe.naver.com/hahadongcamping,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
268,퍼피플,숙박,상촌면 하도대리 25-17,None,0,0.0,None,None,None,None,None
269,바른품미숯불구이원호점,숙박,고아읍 원호리 448-1,None,0,0.0,None,None,None,None,None
270,또랑민박,민박,상촌면 대해리 1035-3,None,0,0.0,None,None,None,None,None
271,풀빛동산펜션,펜션,상촌면 대해리 381,None,0,0.0,None,None,None,None,None


In [32]:
result.head(5)

,place,category,address,tel,review_count,total star,url,user_name,write_date,user_stars,comment
0,B&B모텔,"여관,모텔",부곡동 424-3,054-910-1199,5,1.6,https://kcbnb.modoo.at/,"[YM, 임덕규, 앙기모띠주는나무, 천승민, 김준성]","[2021.06.28., 2020.05.27., 2020.04.15., 2020.0...","[1, 2, 1, 1, 3]","[진짜 불친절합니다, 모텔 옆에 사는 인간이 개를 키우는데 새벽에 개가 갑자기 엄청..."
1,해인산장,"산장,별장",부항면 해인리 266,054-437-1991,1,5.0,http://www.xn--zf4b17j5dm97c.com/,[정발산기슭],[2021.02.06.],[5],[사장님도 친절하시고 고기가 너무 맛있어요. 고기뿐만 아니라 밑반찬도 너무 맛있네요...
2,캠핑뜨락오토캠핑장,오토캠핑장,조마면 강곡리 917-1,010-4114-7122,0,0.0,http://cafe.naver.com/campingdderock,None,None,None,None
3,베네치아,"여관,모텔",감호동 11-2,054-436-1089,1,1.0,None,[.],[2020.09.19.],[1],"[자는데 리모컨 누르지도 않았는데 갑자기 중앙등이 켜지고, 밤새도록 바람소리에 문..."
4,하늘아래첫동네오토캠핑장,오토캠핑장,증산면 금곡리 801,None,0,0.0,http://cafe.naver.com/hahadongcamping,None,None,None,None


## to_csv 

In [33]:
file_path = 'C:/Users/seoyeonho/Desktop/ERC_LAB/김천숙소_수집1차/'
file_name = 'kakaoMaps김천숙소'
result.to_csv('{}{}.csv'.format(file_path,file_name),index=False)

## read_csv

In [29]:
pd.read_csv('{}{}.csv'.format(file_path,file_name))

,Unnamed: 0,place,category,address,tel,review_count,total star,url,user_name,write_date,user_stars,comment
0,0,B&B모텔,"여관,모텔",부곡동 424-3,054-910-1199,5,1.6,https://kcbnb.modoo.at/,"['YM', '임덕규', '앙기모띠주는나무', '천승민', '김준성']","['2021.06.28.', '2020.05.27.', '2020.04.15.', ...","['1', '2', '1', '1', '3']","['진짜 불친절합니다', '모텔 옆에 사는 인간이 개를 키우는데 새벽에 개가 갑자기..."
1,1,해인산장,"산장,별장",부항면 해인리 266,054-437-1991,1,5.0,http://www.xn--zf4b17j5dm97c.com/,['정발산기슭'],['2021.02.06.'],['5'],['사장님도 친절하시고 고기가 너무 맛있어요. 고기뿐만 아니라 밑반찬도 너무 맛있네...
2,2,캠핑뜨락오토캠핑장,오토캠핑장,조마면 강곡리 917-1,010-4114-7122,0,0.0,http://cafe.naver.com/campingdderock,NaN,NaN,NaN,NaN
3,3,베네치아,"여관,모텔",감호동 11-2,054-436-1089,1,1.0,NaN,['.'],['2020.09.19.'],['1'],"['자는데 리모컨 누르지도 않았는데 갑자기 중앙등이 켜지고, 밤새도록 바람소리에 ..."
4,4,하늘아래첫동네오토캠핑장,오토캠핑장,증산면 금곡리 801,NaN,0,0.0,http://cafe.naver.com/hahadongcamping,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
268,268,퍼피플,숙박,상촌면 하도대리 25-17,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN
269,269,바른품미숯불구이원호점,숙박,고아읍 원호리 448-1,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN
270,270,또랑민박,민박,상촌면 대해리 1035-3,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN
271,271,풀빛동산펜션,펜션,상촌면 대해리 381,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN
